In [4]:
 
import pandas as pd
from pathlib import Path
import sqlite3

# Load the uploaded SQLite database
db_path = "../directus-quran/Quran_111.db"
conn = sqlite3.connect(db_path)

# Verify table names
table_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(table_query, conn)

# Debug: Print available tables
print("Available tables in the database:", tables["name"].tolist())

# Read the data only if the table exists
if "TheNobleQuran" in tables["name"].tolist():
    query = """
    SELECT sura_no, sura_name_en, sura_name_roman, ayah_no_surah, ayah_no_quran, ayah_translation_en
    FROM TheNobleQuran
    ORDER BY sura_no, CAST(ayah_no_surah AS INTEGER)
    """
    df = pd.read_sql_query(query, conn)
elif not tables.empty:
    conn.close()
    raise ValueError(f"Table 'TheNobleQuran' does not exist. Available tables: {tables['name'].tolist()}")
else:
    conn.close()
    raise ValueError("No tables exist in the database. Please check the database file.")

conn.close()

# Group by surah and create .mdx content
# Group by surah and create .mdx content
mdx_files = {}
for surah_no, group in df.groupby("sura_no"):
    surah_name_en = group["sura_name_en"].iloc[0]
    surah_name_roman = group["sura_name_roman"].iloc[0]
    
    # Sort the group by verse number to ensure correct order
    # group = group.sort_values('ayah_no_surah')
    
    # Create proper frontmatter with --- on separate lines
    content = "---\n"
    content += f"title: Surah {surah_no} - {surah_name_en} ({surah_name_roman})\n"
    content += f"description: Translation of Surah {surah_no} - {surah_name_en} ({surah_name_roman})\n"
    content += "---\n\n"
    
    # Add heading and content
    # content += f"# Surah {surah_no}: {surah_name_en} ({surah_name_roman})\n\n"
    for _, row in group.iterrows():
        content += f"**{row['ayah_no_surah']}.** {row['ayah_translation_en']}\n\n"
    
    # Pad surah number with leading zeros for correct sorting (e.g., 001, 002, ..., 114)
    filename = f"surah-{str(surah_no).zfill(3)}.mdx"
    mdx_files[filename] = content

# Save .mdx files to a downloadable zip
output_dir = Path("../astro-quran/src/content/docs/surahs")
output_dir.mkdir(exist_ok=True)

for filename, content in mdx_files.items():
    with open(output_dir / filename, "w", encoding="utf-8") as f:
        f.write(content)

# Zip the output directory
# import shutil
# zip_path = "./data/surah-mdx-files.zip"
# shutil.make_archive(zip_path.replace(".zip", ""), 'zip', output_dir)

# print(f"Generated {len(mdx_files)} MDX files with verses in correct numerical order")
# zip_path


Available tables in the database: ['sqlite_sequence', 'directus_migrations', 'directus_folders', 'directus_relations', 'directus_files', 'directus_fields', 'directus_operations', 'directus_notifications', 'directus_translations', 'directus_shares', 'directus_versions', 'directus_revisions', 'directus_users', 'directus_extensions', 'directus_sessions', 'directus_webhooks', 'directus_settings', 'directus_policies', 'directus_permissions', 'directus_access', 'directus_collections', 'directus_dashboards', 'directus_flows', 'directus_panels', 'directus_presets', 'directus_roles', 'directus_comments', 'directus_activity', 'TheNobleQuran']
